<a href="https://colab.research.google.com/github/joscelino/Tensor_Flow-Basico/blob/master/Transfer%C3%AAncia_de_Aprendizado_e_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Etapa 1: Instalação das dependências

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

In [0]:
!pip install tqdm

### Fazendo o download da base de dados de gatos e cachorros

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

## Etapa 2: Pré-processamento

### Importação das bibliotecas

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

### Descompactando a base de dados de gatos e cachorros

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Configurando os caminhos (paths)

In [0]:
dataset_path_new = "./cats_and_dogs_filtered"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [0]:
img_shape = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

In [0]:
base_model.summary()

### Congelando o modelo base

In [0]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [0]:
base_model.output

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output) # Reduzindo a dimensionalidade

In [0]:
global_average_layer

In [0]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = "sigmoid")(global_average_layer)

### Definindo o modelo

In [0]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [0]:
model.summary()

### Compilando o modelo

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001),
              loss="binary_crossentropy", metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

    Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

### Treinando o modelo

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

### Avaliação do modelo de transferência de aprendizagem

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
valid_accuracy

## Fine tuning


Duas questões principais:

- NÃO USE Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico
- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base 

### Descongelando algumas camadas do topo do modelo base

In [0]:
base_model.trainable = True
len(base_model.layers)

In [0]:
fine_tuning_at = 100

In [0]:
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

### Compilando o modelo para fine tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [0]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

### Avaliação do modelo com fine tuning

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
valid_accuracy